In [1]:
import sys
sys.path.append('../..')
from datamart import search, augment
from datamart.utilities.utils import Utils, SEARCH_URL
import pandas as pd
import json

In [2]:
old_df = pd.read_csv("fifa.csv")
print("- READ THE SUPPLY DATASET -\n")
print(old_df.head())

- READ THE SUPPLY DATASET -

   d3mIndex  GameID        Date          Team      Opponent  \
0         0       0  14-06-2018        Russia  Saudi Arabia   
1         1       1  14-06-2018  Saudi Arabia        Russia   
2         2       2  15-06-2018         Egypt       Uruguay   
3         3       3  15-06-2018       Uruguay         Egypt   
4         4       4  15-06-2018       Morocco          Iran   

   Ball Possession %  Off-Target  Blocked  Offsides  Saves  Pass Accuracy %  \
0                 40           3        3         3      0               78   
1                 60           3        3         1      2               86   
2                 43           3        2         1      3               78   
3                 57           6        4         1      3               86   
4                 64           6        4         0      2               86   

   Passes  Distance Covered (Kms)  Yellow & Red  Man of the Match  1st Goal  \
0     306                     118     

In [3]:
with open("fifa_query.json") as f_json:
    query_json = json.load(f_json)
results = search(SEARCH_URL, query_json, old_df)
print("- SEARCH DATAMART BY A DESCRIPTION JSON OBJECT -\n")
print("Returned %d Datasets" % len(results))
for res in results:
    print(res.summary)

- SEARCH DATAMART BY A DESCRIPTION JSON OBJECT -

Returned 1 Datasets
SUMMARY OF THE DATAMART DATASET
        Datamart ID: 127860000
        Score: 85.56142
        Title: FIFA World Cup
        Description: FIFA World Cup
        URL: https://www.football-data.org
        Columns: 
            [0] id 
            [1] season_id 
            [2] season_startDate 
            [3] season_endDate 
            [4] season_currentMatchday 
            [5] utcDate 
            [6] status 
            [7] matchday 
            [8] stage 
            [9] group 
            [10] lastUpdated 
            [11] score_winner 
            [12] score_duration 
            [13] score_fullTime_homeTeam 
            [14] score_fullTime_awayTeam 
            [15] score_halfTime_homeTeam 
            [16] score_halfTime_awayTeam 
            [17] score_extraTime_homeTeam 
            [18] score_extraTime_awayTeam 
            [19] score_penalties_homeTeam 
            [20] score_penalties_awayTeam 
        

In [4]:
new_df = Utils.get_dataset(metadata=results[0].metadata)
print("- MATERIALIZE SEARCH RESULTS -\n")
print(new_df.head())

- MATERIALIZE SEARCH RESULTS -

       id  season_id season_startDate season_endDate  season_currentMatchday  \
0  200000          1       2018-06-14     2018-07-15                       3   
1  200001          1       2018-06-14     2018-07-15                       3   
2  200006          1       2018-06-14     2018-07-15                       3   
3  200007          1       2018-06-14     2018-07-15                       3   
4  200012          1       2018-06-14     2018-07-15                       3   

                utcDate    status  matchday        stage    group  \
0  2018-06-14T15:00:00Z  FINISHED       1.0  GROUP_STAGE  Group A   
1  2018-06-15T12:00:00Z  FINISHED       1.0  GROUP_STAGE  Group A   
2  2018-06-15T15:00:00Z  FINISHED       1.0  GROUP_STAGE  Group B   
3  2018-06-15T18:00:00Z  FINISHED       1.0  GROUP_STAGE  Group B   
4  2018-06-16T10:00:00Z  FINISHED       1.0  GROUP_STAGE  Group C   

            ...           referees_5_nationality referees_6_id  \
0     

In [5]:
target_dataset = results[0]

print("- AUGMENT BY THE SEARCHED RESULT -\n")
result = augment(original_data=old_df, 
                 augment_data=target_dataset, 
                 joining_columns=([ ['Team'],          ['Opponent']      ],
                                  [ ['homeTeam_name'], ['awayTeam_name'] ]) 
                )
print(result.head())

- AUGMENT BY THE SEARCHED RESULT -



TypeError: list indices must be integers or slices, not str